In [1]:
import pathlib
import sys

import joblib
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

sys.path.insert(0, "../utils/")
from data_loader import load_model_data
from model_utils import extract_latent_dims

sys.path.insert(0, "./utils")
from optimizing_utils import model_training

In [88]:
def align_dataframes(
    train: pd.DataFrame, 
    test: pd.DataFrame, 
    val: pd.DataFrame, 
    latent_df: pd.DataFrame
    ):
    """
    Aligns gene expression datasets (train, test, val) with latent representations from latent_df.

    This function merges latent dimension data with RNASeq gene expression datasets based on `SampleID`,
    separates features from target latent dimensions, and removes latent dimensions that contain only zeros.
    Additionally, it combines validation and test features/targets for final evaluation.

    Parameters:
    - train (pd.DataFrame): Training dataset containing RNASeq gene expression data.
    - test (pd.DataFrame): Test dataset containing RNASeq gene expression data.
    - val (pd.DataFrame): Validation dataset containing RNASeq gene expression data.
    - latent_df (pd.DataFrame): Dataframe containing latent dimensions with `ModelID` as the key.

    Returns:
    - X_train_features (pd.DataFrame): Training dataset features (gene expression).
    - X_val_features (pd.DataFrame): Validation dataset features (gene expression).
    - X_test_features (pd.DataFrame): Combined validation & test dataset features (gene expression).
    - y_train (pd.DataFrame): Training dataset target (latent dimensions).
    - y_val (pd.DataFrame): Validation dataset target (latent dimensions).
    - y_test (pd.DataFrame): Combined validation & test dataset target (latent dimensions).
    """
    
    # Identify latent_columns as columns with numeric names
    latent_columns = [col for col in latent_df.columns if col != 'ModelID']

    train = train.merge(latent_df, left_on='SampleID', right_on='ModelID', how='left')
    val = val.merge(latent_df, left_on='SampleID', right_on='ModelID', how='left')
    test = test.merge(latent_df, left_on='SampleID', right_on='ModelID', how='left')

    # Separate the features (RNASeq gene expression) and target (latent dimensions)
    X_train_features = train.drop(columns=['SampleID', 'ModelID'] + latent_columns)
    X_val_features = val.drop(columns=['SampleID', 'ModelID'] + latent_columns)
    X_test_features = test.drop(columns=['SampleID', 'ModelID'] + latent_columns)

    # Set targets as the latent dimensions corresponding to each sample
    y_train = train[latent_columns]
    y_val = val[latent_columns]
    y_test = test[latent_columns]

    # Identify zero-only columns in y_train
    zero_columns = y_train.columns[(y_train == 0).all()]

    # Drop these columns from all y datasets
    y_train = y_train.drop(columns=zero_columns)
    y_val = y_val.drop(columns=zero_columns)
    y_test = y_test.drop(columns=zero_columns)

    y_test = pd.concat([y_val, y_test], axis=0)

    return X_train_features, X_val_features, X_test_features, y_train, y_val, y_test
    

In [29]:
# Load pre-split RNASeq data
train = pd.read_parquet("./data/RNASeq_train_zscored.parquet")
val = pd.read_parquet("./data/RNASeq_val_zscored.parquet")
test = pd.read_parquet("./data/RNASeq_test_zscored.parquet")

In [4]:
data_directory = pathlib.Path("../0.data-download/data").resolve()
dependency_file = pathlib.Path(f"{data_directory}/CRISPRGeneEffect.parquet").resolve()
gene_dict_file = pathlib.Path(f"{data_directory}/CRISPR_gene_dictionary.parquet").resolve()

In [5]:
# Load metadata
metadata_df_dir = pathlib.Path("../0.data-download/data/metadata_df.parquet").resolve()
metadata = pd.read_parquet(metadata_df_dir)
print(metadata.shape)

#Load dependency data
dependency_df, gene_dict_df = load_model_data(dependency_file, gene_dict_file)
dependency_df.head()

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply the scaler to the numeric columns
dependency_df[dependency_df.select_dtypes(include=['float64', 'int']).columns] = scaler.fit_transform(
    dependency_df.select_dtypes(include=['float64', 'int'])
)

(958, 3)
(1150, 18444)


In [6]:
train_and_test_subbed_dir = pathlib.Path("../0.data-download/data/train_and_test_subbed.parquet").resolve()
train_and_test_subbed = pd.read_parquet(train_and_test_subbed_dir)

# Convert DataFrame to NumPy and then Tensor
train_test_array = train_and_test_subbed.to_numpy()
train_test_tensor = torch.tensor(train_test_array, dtype=torch.float32)

#Create TensorDataset and DataLoader
tensor_dataset = TensorDataset(train_test_tensor)
train_and_test_subbed_loader = DataLoader(tensor_dataset, batch_size=32, shuffle=False)

In [47]:
# Define the location of the saved models and output directory for results
model_save_dir = pathlib.Path("../4.gene-expression-signatures/saved_models").resolve()
output_dir = pathlib.Path("results").resolve()
output_dir.mkdir(parents=True, exist_ok=True)

# File to store the combined correlation results
final_test_results_file = output_dir / "test_r2.parquet"
final_test_predictions_file = output_dir /  "test_preds.parquet"
final_test_actual_file = output_dir /  "pred_vs_actual.parquet"

In [109]:
# Initialize lists to hold test results and predictions
all_test_results = []
all_test_predictions = []

# Load existing results if available
if pathlib.Path(final_test_results_file).exists():
    final_test_results_df = pd.read_parquet(final_test_results_file)
    final_test_predictions_df = pd.read_parquet(final_test_predictions_file)
    all_test_results.append(final_test_results_df)
    all_test_predictions.append(final_test_predictions_df)
    print(f"Loaded existing test results from {final_test_results_file}")
    print(f"Existing test results loaded: {final_test_results_df.shape}")
    print(f"Existing test predictions loaded: {final_test_predictions_df.shape}")
else:
    final_test_results_df = pd.DataFrame(columns=['model','latent_dimension', 'z_dimension', 'init', 'R2_score'])
    final_test_predictions_df = pd.DataFrame(columns=['model', 'latent_dimension', 'z_dimension', 'init', 'type']) 
    print("Starting with empty DataFrames.")

# Iterate over all files in the saved_models directory
for model_file in model_save_dir.glob("*.joblib"):
    model_file_name = model_file.stem
    try:
        parts = model_file_name.split("_")
        model_name = parts[0]
        num_components = int(parts[3])
        init = int(parts[7])
        seed = int(parts[9])
    except (IndexError, ValueError):
        print(f"Skipping file {model_file} due to unexpected filename format.")
        continue

    expected_z_dimensions = {int(f"{i}") for i in range(num_components)}

    # Get already processed z_dimensions
    processed_z_dimensions = set(final_test_predictions_df[
        (final_test_predictions_df['model'] == model_name) &
        (final_test_predictions_df['init'] == init) &
        (final_test_predictions_df['latent_dimension'] == num_components)
    ]['z_dimension'].unique())

    processed_z_dimensions = {int(col.lstrip('z_')) for col in processed_z_dimensions}

    # Determine which z_dimensions still need processing
    missing_z_dimensions = expected_z_dimensions - processed_z_dimensions

    if not missing_z_dimensions:
        print(f"Skipping {model_name} init {init} with {num_components} dimensions. All z_dimensions are processed.")
        continue
    else:
        print(f"Processing missing z_dimensions: {missing_z_dimensions}")

    # Load the model
    print(f"Loading model from {model_file}")
    try:
        model = joblib.load(model_file)
    except Exception as e:
        print(f"Failed to load model from {model_file}: {e}")
        continue

    # Extract latent dimensions
    latent_df = extract_latent_dims(model_name, model, dependency_df, train_and_test_subbed_loader, metadata)

    # Ensure missing_z_dimensions is of type set of integers
    missing_z_dimensions = {int(dim) for dim in missing_z_dimensions}  # Make sure the missing_z_dimensions are integers

    # Ensure latent_df.columns are integers for comparison
    latent_df.columns = [int(col) if isinstance(col, str) and col.isdigit() else col for col in latent_df.columns]

    # Filter latent_df to only include missing z_dimensions (ensure the column names are integers)
    latent_df = latent_df[[col for col in latent_df.columns if (isinstance(col, int) and col in missing_z_dimensions) or col == 'ModelID']]

    # Align dataframes for training and testing
    x_train, x_val, x_test, y_train, y_val, y_test = align_dataframes(train, test, val, latent_df)

    # Perform model training and prediction only for missing z_dimensions
    test_results_df, test_predictions_df = model_training(
        x_train, x_val, x_test, y_train, y_val, y_test, model_name, num_components, init, seed
    )

    # Add metadata columns for test results and predictions
    test_results_df = test_results_df.assign(init=init, seed=seed)
    test_predictions_df = test_predictions_df.assign(init=init, seed=seed)

    # Append only new results to avoid duplication
    all_test_results.append(test_results_df)
    all_test_predictions.append(test_predictions_df)

    # Incrementally update final dataframes
    final_test_results_df = pd.concat(all_test_results, ignore_index=True).drop_duplicates()
    final_test_predictions_df = pd.concat(all_test_predictions, ignore_index=True).drop_duplicates()

    # Clean up z_dimension naming
    final_test_results_df['z_dimension'] = final_test_results_df['z_dimension'].astype(str)
    final_test_predictions_df['z_dimension'] = final_test_predictions_df['z_dimension'].astype(str)
    final_test_results_df['z_dimension'] = final_test_results_df['z_dimension'].str.replace(r'^z_z_', 'z_', regex=True)
    final_test_predictions_df['z_dimension'] = final_test_predictions_df['z_dimension'].str.replace(r'^z_z_', 'z_', regex=True)

    # Save results and predictions
    final_test_results_df.to_parquet(final_test_results_file)
    final_test_predictions_df.to_parquet(final_test_predictions_file)

print(f"Saved test results to {final_test_results_file}")
print(f"Saved test predictions to {final_test_predictions_file}")


Loaded existing test results from results/test_r2.parquet
Existing test results loaded: (15309, 6)
Existing test predictions loaded: (30607, 274)
Skipping betatcvae init 2 with 5 dimensions. All z_dimensions are processed.
Processing missing z_dimensions: {1}
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_14_trial_42_init_4_seed_1154917792.joblib
Skipping betatcvae init 1 with 20 dimensions. All z_dimensions are processed.
Skipping betavae init 4 with 3 dimensions. All z_dimensions are processed.
Processing missing z_dimensions: {129, 132, 8, 13, 142, 148, 27, 28, 31, 33, 46, 49, 52, 55, 67, 69, 82, 86, 97, 100, 101, 102, 105, 106, 109, 122, 123}
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_150_trial_30_init_1_seed_1559165653.joblib
Skipping pca init 0 with 35 dimensions. All z_dimensions are processed.
Skipping ica init 0 with 50 dimensions. All z_dimensions are processed.
Processing missing z_dimensions: {

/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator NMF from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but NMF was fitted without feature names
  warnings.warn(


Skipping ica init 0 with 35 dimensions. All z_dimensions are processed.
Processing missing z_dimensions: {0, 1, 2, 4, 6, 8, 16, 20, 21, 25, 26, 27, 30, 31, 32, 35, 40, 41, 42, 50, 53, 54, 65, 67}
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_70_trial_22_init_2_seed_1709873177.joblib
Skipping nmf init 0 with 18 dimensions. All z_dimensions are processed.
Processing missing z_dimensions: {33, 2, 34, 37, 8, 40, 59, 12, 17, 18, 19, 20, 50, 53, 57, 27, 28, 31}
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_60_trial_26_init_4_seed_4073875038.joblib
Skipping pca init 0 with 14 dimensions. All z_dimensions are processed.
Skipping betavae init 2 with 5 dimensions. All z_dimensions are processed.
Skipping pca init 0 with 8 dimensions. All z_dimensions are processed.
Skipping betatcvae init 1 with 12 dimensions. All z_dimensions are processed.
Skipping betatcvae init 4 with 35 dimensions. All z_dimensions are processed.

/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator NMF from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but NMF was fitted without feature names
  warnings.warn(


Processing missing z_dimensions: {132, 10, 11, 12, 141, 17, 18, 19, 145, 147, 24, 25, 154, 28, 158, 36, 166, 171, 44, 49, 55, 56, 189, 63, 67, 68, 196, 73, 74, 76, 85, 89, 93, 109, 112, 113}
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_200_trial_13_init_3_seed_3783068897.joblib
Processing missing z_dimensions: {2, 130, 5, 6, 134, 136, 138, 139, 140, 143, 144, 147, 148, 22, 23, 25, 26, 28, 156, 157, 36, 38, 170, 171, 48, 52, 180, 54, 184, 185, 189, 190, 198, 199, 78, 83, 84, 90, 103, 107, 108, 112, 114, 118, 126}
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_200_trial_27_init_2_seed_1709873177.joblib
Processing missing z_dimensions: {2, 3, 4, 7, 8}
Loading model from ../4.gene-expression-signatures/saved_models/vanillavae_latent_dims_10_trial_36_init_4_seed_1154917792.joblib
Skipping vanillavae init 1 with 7 dimensions. All z_dimensions are processed.
Processing missing z_dimensions: {0, 5, 6, 7, 9, 13, 1

...

Processing latent dimension: 0
Loading pre-trained model for 0 from joblib/elasticnet_pca_dims_100_z_0_init_0_seed_0.joblib
Processing latent dimension: 1
Loading pre-trained model for 1 from joblib/elasticnet_pca_dims_100_z_1_init_0_seed_0.joblib
Processing latent dimension: 2
Loading pre-trained model for 2 from joblib/elasticnet_pca_dims_100_z_2_init_0_seed_0.joblib
Processing latent dimension: 3
Loading pre-trained model for 3 from joblib/elasticnet_pca_dims_100_z_3_init_0_seed_0.joblib
Processing latent dimension: 4
Loading pre-trained model for 4 from joblib/elasticnet_pca_dims_100_z_4_init_0_seed_0.joblib
Processing latent dimension: 5
Loading pre-trained model for 5 from joblib/elasticnet_pca_dims_100_z_5_init_0_seed_0.joblib
Processing latent dimension: 6
Loading pre-trained model for 6 from joblib/elasticnet_pca_dims_100_z_6_init_0_seed_0.joblib
Processing latent dimension: 7
Loading pre-trained model for 7 from joblib/elasticnet_pca_dims_100_z_7_init_0_seed_0.joblib
Processi

/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


Processing latent dimension: 1
Loading pre-trained model for 1 from joblib/elasticnet_pca_dims_7_z_1_init_0_seed_0.joblib
Processing latent dimension: 2
Loading pre-trained model for 2 from joblib/elasticnet_pca_dims_7_z_2_init_0_seed_0.joblib
Processing latent dimension: 3
Loading pre-trained model for 3 from joblib/elasticnet_pca_dims_7_z_3_init_0_seed_0.joblib
Processing latent dimension: 4
Loading pre-trained model for 4 from joblib/elasticnet_pca_dims_7_z_4_init_0_seed_0.joblib
Processing latent dimension: 5
Loading pre-trained model for 5 from joblib/elasticnet_pca_dims_7_z_5_init_0_seed_0.joblib
Processing latent dimension: 6
Loading pre-trained model for 6 from joblib/elasticnet_pca_dims_7_z_6_init_0_seed_0.joblib
Testing for latent dimension: 0
Loading model for testing from joblib/elasticnet_pca_dims_7_z_0_init_0_seed_0.joblib
Test R² score for 0: -0.1529
Testing for latent dimension: 1
Loading model for testing from joblib/elasticnet_pca_dims_7_z_1_init_0_seed_0.joblib
Test 

/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator NMF from version 1.5.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/juliacurd/anaconda3/envs/gene_dependency_representations_real/lib/python3.12/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but NMF was fitted without feature names
  warnings.warn(


Processing latent dimension: 0
Loading pre-trained model for 0 from joblib/elasticnet_nmf_dims_2_z_0_init_0_seed_0.joblib
Processing latent dimension: 1
Loading pre-trained model for 1 from joblib/elasticnet_nmf_dims_2_z_1_init_0_seed_0.joblib
Testing for latent dimension: 0
Loading model for testing from joblib/elasticnet_nmf_dims_2_z_0_init_0_seed_0.joblib
Test R² score for 0: -0.0148
Testing for latent dimension: 1
Loading model for testing from joblib/elasticnet_nmf_dims_2_z_1_init_0_seed_0.joblib
Test R² score for 1: -0.5055
Processing missing z_dimensions: {0, 1, 2}
Loading model from ../4.gene-expression-signatures/saved_models/betavae_latent_dims_3_trial_11_init_0_seed_950231792.joblib
Processing latent dimension: 0
Loading pre-trained model for 0 from joblib/elasticnet_betavae_dims_3_z_0_init_0_seed_950231792.joblib
Processing latent dimension: 1
Loading pre-trained model for 1 from joblib/elasticnet_betavae_dims_3_z_1_init_0_seed_950231792.joblib
Testing for latent dimension:

In [111]:
# Sort test results for R² in descending order and get the top 10 latent dimensions
top_20_r2 = final_test_results_df.sort_values(by='R2_score', ascending=False).head(20)
print("Top 20 R² scores:")
print(top_20_r2)

Top 20 R² scores:
       latent_dimension z_dimension  R2_score       model  init        seed
979               150.0       z_140  0.781724  vanillavae     3  3783068897
14427              80.0        z_29  0.781454     betavae     3  3783068897
65                150.0        z_57  0.772158  vanillavae     1  1559165653
6447              150.0        z_18  0.768828  vanillavae     2  1709873177
8749               50.0        z_49  0.768274     betavae     1  1559165653
940               150.0        z_58  0.757152  vanillavae     3  3783068897
13322             200.0        z_51  0.755208     betavae     1  1559165653
6505              150.0        z_93  0.754547  vanillavae     2  1709873177
2720              200.0        z_99  0.753200  vanillavae     3  3783068897
6348               30.0        z_11  0.752004  vanillavae     3  2176741505
15593             150.0       z_113  0.751408     betavae     0  1590696690
14275              18.0        z_14  0.749718     betavae     3  21767

In [112]:
print(final_test_predictions_df.shape)
final_test_predictions_df.head()

(35493, 274)


,model,latent_dimension,z_dimension,type,0,1,2,3,4,5,...,260,261,262,263,264,265,266,267,init,seed
0,betatcvae,5.0,z_0,predicted,0.197768,0.149162,0.136214,0.194249,0.337518,0.185987,...,0.151966,0.159155,0.332227,0.052263,0.229891,0.162828,0.175713,0.140719,2,1999415996
1,betatcvae,5.0,z_0,actual,0.173075,0.158538,0.005038,0.178626,0.358158,0.334751,...,0.161952,0.119415,0.474365,0.134533,-0.100073,0.090227,0.210525,0.287258,2,1999415996
2,betatcvae,5.0,z_1,predicted,-0.579397,-0.359742,-0.329709,-0.563084,-0.707539,-0.420481,...,-0.418177,-0.393277,-0.696133,-0.251096,-0.535756,-0.475505,-0.441577,-0.399799,2,1999415996
3,betatcvae,5.0,z_1,actual,-0.589149,-0.334081,-0.140173,-0.414100,-0.686831,-0.685121,...,-0.358154,-0.326517,-0.932266,-0.270638,-0.042393,-0.262340,-0.507877,-0.692733,2,1999415996
4,betatcvae,5.0,z_2,predicted,0.230649,0.227100,0.237732,0.268510,-0.007863,0.142058,...,0.204281,0.173011,0.070251,0.305940,0.142347,0.255937,0.203420,0.257086,2,1999415996
